In [1]:
import os
import numpy as np
import scipy.stats as stats
from scipy.stats._stats import _kendall_dis
import statistics
import time
# import warnings

## Evaluation metrics

In [2]:
# Evaluation metrics
def top_n_accuracy(pred_val, gt_val, n: int):
    tmp_pred = pred_val.copy()
    tmp_gt_val = gt_val.copy()
    tmp_pred.sort(key=lambda element: element[1], reverse=True)
    tmp_gt_val.sort(key=lambda element: element[1], reverse=True)
    
    pred, gt = [], []
    for i in range(int(len(tmp_pred)*n/100)):
        pred.append(tmp_pred[i][0])
        gt.append(tmp_gt_val[i][0])

    return float(len(set(gt)&set(pred)) / len(pred))

def kadabra_top_n_accuracy(pred_val, gt_val, n: int):
    # tmp_pred = pred_val.copy()
    tmp_gt_val = gt_val.copy()
    # tmp_pred.sort(key=lambda element: element[1], reverse=True)
    tmp_gt_val.sort(key=lambda element: element[1], reverse=True)
    
    pred, gt = [], []
    for i in range(n):
        pred.append(int(pred_val[i]))
        gt.append(int(tmp_gt_val[i][0]))

    return float(len(set(gt)&set(pred)) / len(pred))

def Kendall_tau_distance(pred_val, gt_val):
    pred_bc_val, gt_bc_val = [], []
    for i in pred_val:
        pred_bc_val.append(i[1])
    for i in gt_val:
        gt_bc_val.append(i[1])

    tau, _ = stats.kendalltau(pred_bc_val, gt_bc_val)
    
    return float(tau)

## Kadabra

In [3]:
os.chdir('./kadabra/')

In [4]:
os.getcwd()

'/home/lin-chia/Desktop/kadabra'

### Synthetic data

#### Computing running time

In [10]:
wall_clock_time_list = []
for i in range(30):
    t0 = time.time()
    input_file = '../hw1_data/Synthetic/5000/'+str(i)+'.txt'
    cmd = './kadabra 0.001 0.1 '+input_file
    os.system(cmd)
    wall_clock_time = time.time() - t0
    wall_clock_time_list.append(wall_clock_time)
    
print('avg excute time',sum(wall_clock_time_list)/len(wall_clock_time_list))
print('stdev excute time',statistics.stdev(wall_clock_time_list))

Undirected graph
Number of nodes: 5000
Number of edges: 39964

Finished after 1587641 iterations.
Edges visited: 441280094
Average edges visited: 277
Total time: 0.848279
Time bfs: 0.708602
Time critical: 0.0509996
Time compute finished: 0.0414002
(Printing thread: 0)
Maximum confidence interval: 0.000999972
Undirected graph
Number of nodes: 5000
Number of edges: 39962

Finished after 1540671 iterations.
Edges visited: 431655606
Average edges visited: 280
Total time: 0.956619
Time bfs: 0.800691
Time critical: 0.058224
Time compute finished: 0.0470524
(Printing thread: 0)
Maximum confidence interval: 0.000999958
Undirected graph
Number of nodes: 5000
Number of edges: 39960

Finished after 1568688 iterations.
Edges visited: 436952590
Average edges visited: 278
Total time: 0.971778
Time bfs: 0.812547
Time critical: 0.0549366
Time compute finished: 0.0505261
(Printing thread: 0)
Maximum confidence interval: 0.000999992
Undirected graph
Number of nodes: 5000
Number of edges: 39964

Finished

#### Output Top-k% betweenness centrality

In [98]:
for i in range(30):
    for k in [1, 5, 10]:
        k_param = 5000*k // 100 # Top-K% node
        input_file = '../hw1_data/Synthetic/5000/'+str(i)+'.txt'
        output_file = '../baseline_model/kadabra/Synthetic/'+str(i)+'_top'+str(k)+'_score.txt'
        cmd = './kadabra -k '+str(k_param)+' 0.001 0.1 '+input_file+' > '+output_file
        os.system(cmd)

### Real-world data

#### Computing running time

In [102]:
t0 = time.time()
# k_param = 1134890*k//100
input_file = '../hw1_data/youtube/com-youtube.txt'
# output_file = '../baseline_model/kadabra/Youtube/com-youtube_score.txt'
cmd = './kadabra 0.001 0.1 '+input_file
os.system(cmd)
wall_clock_time = time.time() - t0

print('excute time: ', wall_clock_time)

Undirected graph
Number of nodes: 1134890
Number of edges: 5975248

Situation after 448125 iterations.
Edges visited: 4225355130
Average edges visited: 9428
Total time: 60.0007
Time bfs: 58.6097
Time critical: 0.144347
Time compute finished: 0.806048
(Printing thread: 3)
Maximum confidence interval: 0.00583006

Situation after 935960 iterations.
Edges visited: 8474358378
Average edges visited: 9054
Total time: 120.001
Time bfs: 117.61
Time critical: 0.301937
Time compute finished: 1.62957
(Printing thread: 7)
Maximum confidence interval: 0.00276782

Situation after 1421568 iterations.
Edges visited: 12723096948
Average edges visited: 8950
Total time: 180.002
Time bfs: 176.577
Time critical: 0.46845
Time compute finished: 2.49103
(Printing thread: 5)
Maximum confidence interval: 0.00181728

Situation after 1907444 iterations.
Edges visited: 16967626713
Average edges visited: 8895
Total time: 240.002
Time bfs: 235.544
Time critical: 0.641661
Time compute finished: 3.33534
(Printing threa

#### Output Top-k% betweenness centrality

In [6]:
for k in [1, 5, 10]:
    k_param = 1134890*k//100
    input_file = '../hw1_data/youtube/com-youtube.txt'
    output_file = '../baseline_model/kadabra/Youtube/'+'top'+str(k)+'_com-youtube_score.txt'
    cmd = './kadabra -k '+str(k_param)+' 0.001 0.1 '+input_file+' > '+output_file
    os.system(cmd)

11348


### Testing

#### Synthetic data

In [99]:
top1_list = []
top5_list = []
top10_list = []
for i in range(30):
    for k in [1, 5, 10]:
        num_node = 5000*k // 100
        pred_val_path = '../baseline_model/kadabra/' + str(i) + '_top' + str(k) + '_score.txt'
        gt_val_path = '../hw1_data/Synthetic/5000/' + str(i) + '_score.txt'
        f_pred = open(pred_val_path, mode='r')
        f_gt = open(gt_val_path, mode='r')
        tmp_pred_list, pred_list, gt_list = [], [], []
        for line in f_pred:
            # Get the recorded score line
            if line[:5] == '     ':
                tmp_pred_list.append(line[:-1])
        for j in tmp_pred_list:
            ind = j.split()[1]
            pred_list.append(ind)

        for line in f_gt:
            ind, score = line.split()
            gt_list.append([int(ind), float(score)])
        f_pred.close()
        f_gt.close()

        if k == 1:
            top1_list.append(kadabra_top_n_accuracy(pred_list[:num_node], gt_list[:num_node], num_node))
        elif k == 5:
            top5_list.append(kadabra_top_n_accuracy(pred_list[:num_node], gt_list[:num_node], num_node))
        elif k == 10:
            top10_list.append(kadabra_top_n_accuracy(pred_list[:num_node], gt_list[:num_node], num_node))

print("top-1%:", sum(top1_list) / len(top1_list),'+-',statistics.stdev(top1_list))
print("top-5%:", sum(top5_list) / len(top5_list),'+-',statistics.stdev(top5_list))
print("top-10%:", sum(top10_list) / len(top10_list),'+-',statistics.stdev(top10_list))

top-1%: 0.7066666666666667 +- 0.048801733528659595
top-5%: 0.7136000000000001 +- 0.01775231893545762
top-10%: 0.7174666666666666 +- 0.012022201684106518


#### Real-world

In [18]:
top1_list = []
top5_list = []
top10_list = []
for k in [1, 5, 10]:
    num_node = 1134890*k // 100
    pred_val_path = '../baseline_model/kadabra/Youtube/top' + str(k) + '_com-youtube_score.txt'
    gt_val_path = '../hw1_data/youtube/com-youtube_score.txt'
    f_pred = open(pred_val_path, mode='r')
    f_gt = open(gt_val_path, mode='r')
    tmp_pred_list, pred_list, gt_list = [], [], []
    record_signal = 0
    record_count = 0
    for line in f_pred:
        if record_signal == 1:
            tmp_pred_list.append(line[:-1])
            record_count += 1
        if record_count == num_node:
            break
        # Get the recorded score line signal.
        if line[:20] == '(Printing thread: 0)':
            record_signal = 1
    for j in tmp_pred_list:
        ind = j.split()  
        ind = list(filter(None,ind))
        if '?' in ind:
            ind.remove('?')
        pred_list.append(int(ind[1]))
    for line in f_gt:
        ind, score = line.split(':')
        score = score.split()[0]
        gt_list.append([int(ind), float(score)])
    f_pred.close()
    f_gt.close()

    if k == 1:
        print('Top-1%:{}'.format(kadabra_top_n_accuracy(pred_list[:num_node], gt_list[:num_node], num_node)))
    elif k == 5:
        print('Top-5%:{}'.format(kadabra_top_n_accuracy(pred_list[:num_node], gt_list[:num_node], num_node)))
    elif k == 10:
        print('Top-10%:{}'.format(kadabra_top_n_accuracy(pred_list[:num_node], gt_list[:num_node], num_node)))


Top-1%:0.19095875925273176
Top-5%:0.3078034682080925
Top-10%:0.34601591343654453
